In [66]:
import sys
sys.path.insert(0,'..')

from src.models import *
from src.loss_functions import *
from src.noise import *
from src.metrics import *
from src.plotting import *
from src.generate_data import *
from src.real_data import *

import sklearn
import pandas as pd

from scipy.stats import bernoulli

from operator import xor
from sklearn.preprocessing import StandardScaler

import pickle as pkl

In [113]:
def simulate_noise_and_train_model(m, max_iter, X_train, y_train, X_test, y_test, p_y_x_dict, noise_type = "class_independent", uncertainty_type="backward",  model_type = "LR" , fixed_class=0, fixed_noise=0.2, T_true = None, T_est = None, batch_size = 512, base_seed = 2024, epsilon = 0.25):
    
    if uncertainty_type == "forward":
        loss_types = ["Ours", "BCE", "backward", "forward"]
        y_vec = y_train
    else: # backward
        loss_types = ["Ours"]

        #Initial Noise Draw
        u_vec = get_u(y_train, T = T_true, seed= base_seed, noise_type = noise_type)
        y_vec = flip_labels(y_train, u_vec) #XOR

    metrics = MetricsStorage(loss_types)

    preds_train_dict = {loss: [] for loss in loss_types}
    preds_test_dict = {loss: [] for loss in loss_types}

    typical_count = 0

    
    for seed in tqdm(range(1, max_iter+1)):
        if uncertainty_type == "forward":
            # Using a forward model, so get u directly
            u_vec = get_u(y_vec, T = T_true, seed= seed, noise_type = noise_type)
        else:
            u_vec = infer_u(y_vec, noise_type = noise_type, p_y_x_dict = p_y_x_dict,  T = T_est , seed=seed)

        typical_flag, difference = is_typical(u_vec, p_y_x_dict,  T = T_est, y_vec = y_vec, noise_type = noise_type, uncertainty_type = uncertainty_type, epsilon = epsilon)

        if not typical_flag: 
            continue

        flipped_labels = flip_labels(y_vec, u_vec)

        if uncertainty_type == "forward":
            for loss in loss_types:
                if loss == "Ours":
                    model,  (train_acc,
                        test_acc,
                        train_probs,
                        test_probs,
                        train_loss,
                        test_loss,
                        train_preds,
                        test_preds
                        ) = train_model_ours(X_train, flipped_labels, X_test, y_test, seed = 2024, model_type=model_type)

                    preds_train_dict[loss].append(train_preds)
                    preds_test_dict[loss].append(test_preds)

                    metrics.add_metric(loss, "noisy_train_loss", train_loss)
                    metrics.add_metric(loss, "noisy_train_acc", train_acc*100)
                    metrics.add_metric(loss, "clean_test_loss", test_loss)
                    metrics.add_metric(loss, "clean_test_acc", test_acc*100)
                    metrics.add_metric(loss, "typical_difference", difference)
                    metrics.add_metric(loss, "preds_train", train_preds)
                    metrics.add_metric(loss, "preds_test", test_preds)
                    metrics.add_metric(loss, "train_probs", train_probs)
                    metrics.add_metric(loss, "test_probs", test_probs)

                else:
                    model,  (noisy_train_loss,
                            clean_train_loss, 
                            noisy_train_acc,
                            clean_train_acc,
                            train_probs,
                            clean_test_loss, 
                            clean_test_acc,
                            test_probs
                            ) = train_model(X_train, y_train, flipped_labels,  X_test, y_test,  T = T_est, seed=2024, num_epochs=25, batch_size=batch_size, model_type = model_type, correction_type=loss)

                    preds_train = (train_probs > 0.5).astype(int)
                    preds_test = (test_probs > 0.5).astype(int)

                    preds_train_dict[loss].append(preds_train)
                    preds_test_dict[loss].append(preds_test)

                    metrics.add_metric(loss, "noisy_train_loss", noisy_train_loss)
                    metrics.add_metric(loss, "clean_train_loss", clean_train_loss)
                    metrics.add_metric(loss, "noisy_train_acc", noisy_train_acc*100)
                    metrics.add_metric(loss, "clean_train_acc", clean_train_acc*100)
                    metrics.add_metric(loss, "clean_test_loss", clean_test_loss)
                    metrics.add_metric(loss, "clean_test_acc", clean_test_acc*100)
                    metrics.add_metric(loss, "flip_frequency", sum(u_vec)/len(u_vec))
                    metrics.add_metric(loss, "typical_difference", difference)
                    metrics.add_metric(loss, "preds_train", preds_train)
                    metrics.add_metric(loss, "preds_test", preds_test)
                    metrics.add_metric(loss, "train_probs", train_probs)
                    metrics.add_metric(loss, "test_probs", test_probs)

        else: #backward_sk
            
            for loss in loss_types:
                model,  (train_acc,
                        test_acc,
                        train_probs,
                        test_probs,
                        train_loss,
                        test_loss,
                        train_preds,
                        test_preds
                        ) = train_model_ours(X_train, flipped_labels, X_test, y_test, seed = 2024, model_type=model_type)

                preds_train_dict[loss].append(train_preds)
                preds_test_dict[loss].append(test_preds)

                metrics.add_metric(loss, "train_loss", train_loss)
                metrics.add_metric(loss, "train_acc", train_acc*100)
                metrics.add_metric(loss, "test_loss", test_loss)
                metrics.add_metric(loss, "test_acc", test_acc*100)
                metrics.add_metric(loss, "typical_difference", difference)
                metrics.add_metric(loss, "preds_train", train_preds)
                metrics.add_metric(loss, "preds_test", test_preds)
                metrics.add_metric(loss, "train_probs", train_probs)
                metrics.add_metric(loss, "test_probs", test_probs)

        typical_count += 1

        if typical_count == m:
            break

    for loss in loss_types:
        typical_rate = typical_count / seed
        print("Typical Rate: ", typical_rate)
        metrics.add_metric(loss, "typical_rate", typical_rate)

        predictions_train = np.array(preds_train_dict[loss])

        predictions_test = np.array(preds_test_dict[loss])

        try:
            regret_train = calculate_error_rate(predictions_train, y_train)
            disagreement_train = estimate_disagreement(predictions_train)

            regret_test = calculate_error_rate(predictions_test, y_test)
            disagreement_test = estimate_disagreement(predictions_test)

        except:
            print("Error: Could not get Disagreement Metrics")
            continue

        for i, item in enumerate(X_train):
            metrics.add_metric(loss, "regret_train", regret_train[i])
            metrics.add_metric(loss, "disagreement_train", disagreement_train[i])

        for i, item in enumerate(X_test):

            metrics.add_metric(loss, "regret_test", regret_test[i])
            metrics.add_metric(loss, "disagreement_test", disagreement_test[i])

    print("DONE")
    return metrics


def train_model(X_train, y_train, yn_train, X_test, y_test, T,  seed, num_epochs=25, batch_size = 512, correction_type="forward", model_type = "LR"):
    # Check if GPU is available and set the default device accordingly
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Convert to PyTorch tensors and move them to the device
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    yn_train = torch.tensor(yn_train, dtype=torch.long).to(device)

    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # Create DataLoader for mini-batch SGD
    train_data = TensorDataset(X_train, yn_train, y_train)
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

    if model_type == "LR":
        # Initialize the model and move it to the device
        model = LogisticRegression(X_train.shape[1]).to(device)
    else:
        # Initialize the model and move it to the device
        model = NeuralNet(X_train.shape[1]).to(device)

    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(model.parameters(), lr=0.01)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    if correction_type in ['backward', 'forward']:
        #elapsed = timeit.default_timer() - start_time
        #T = torch.tensor(np.array([T_dict[g.item()] for g in group]), dtype=torch.float32).to(device)
        #print("T_dict ", elapsed)

        T = torch.tensor(T).to(device)

    # Train the model
    for epoch in (range(num_epochs)):
        for features, noisy_labels, clean_labels in train_loader:
            

            # Move features and labels to the device
            features, noisy_labels, clean_labels = features.to(device), noisy_labels.to(device), clean_labels.to(device)
            
            # Forward pass
            outputs = model(features)

            if correction_type == 'forward':
                noisy_loss = forward_loss(outputs, noisy_labels, T, device)

                #elapsed = timeit.default_timer() - start_time
                #print("Forward ", elapsed)

            elif correction_type == 'backward':
                noisy_loss = backward_loss(outputs, noisy_labels, T, device)

                #elapsed = timeit.default_timer() - start_time
                #print("Backward ", elapsed)

            else:
                noisy_loss = criterion(outputs, noisy_labels)
                
                #elapsed = timeit.default_timer() - start_time
                #print("BCE ", elapsed)
            #print(noisy_loss.item())
            # Backward pass and optimization
            optimizer.zero_grad()
            noisy_loss.backward()
            optimizer.step()
    
    train_outputs = model(X_train)
    test_outputs = model(X_test)

    #Get final train losses
    if correction_type == 'forward':
        
        #T_train = torch.tensor(np.array([T_dict[g.item()] for g in group_train]), dtype=torch.float32).to(device)
        #T_test = torch.tensor(np.array([T_dict[g.item()] for g in group_test]), dtype=torch.float32).to(device)
        
        noisy_train_loss = forward_loss(train_outputs, yn_train, T, device).item()
        clean_train_loss = forward_loss(train_outputs, y_train, T, device).item()
        
        clean_test_loss = forward_loss(test_outputs, y_test, T, device).item()
        
    elif correction_type == 'backward':
        #T_train = torch.tensor(np.array([T_dict[g.item()] for g in group_train]), dtype=torch.float32).to(device)
        #T_test = torch.tensor(np.array([T_dict[g.item()] for g in group_test]), dtype=torch.float32).to(device)
        
        noisy_train_loss = backward_loss(train_outputs, yn_train, T, device).item()
        clean_train_loss = backward_loss(train_outputs, y_train, T, device).item()
        
        clean_test_loss = backward_loss(test_outputs, y_test, T, device).item()
    else:
        noisy_train_loss = criterion(train_outputs, yn_train).item()

        clean_train_loss = criterion(train_outputs, y_train).item()
        clean_test_loss = criterion(test_outputs, y_test).item()


    # Evaluate the model
    with torch.no_grad():

        _, predicted = torch.max(test_outputs.data, 1)
        # Move the predictions back to the CPU for sklearn accuracy calculation
        clean_test_acc = accuracy_score(y_test.cpu().numpy(), predicted.cpu().numpy())
        test_probs = torch.softmax(test_outputs, dim=1)[:, 1].cpu().numpy()

        _, predicted = torch.max(train_outputs.data, 1)
        # Move the predictions back to the CPU for sklearn accuracy calculation
        clean_train_acc = accuracy_score(y_train.cpu().numpy(), predicted.cpu().numpy())
        noisy_train_acc = accuracy_score(yn_train.cpu().numpy(), predicted.cpu().numpy())
        train_probs = torch.softmax(train_outputs, dim=1)[:, 1].cpu().numpy()

    results = (noisy_train_loss,
                clean_train_loss, 
                noisy_train_acc,
                clean_train_acc,
                train_probs,
                clean_test_loss, 
                clean_test_acc,
                test_probs
                )
    return model, results


In [114]:
dataset = "cshock_mimic"

X_train, X_test, y_train, y_test, group_train, group_test = load_dataset_splits(dataset, group = "age")

    
print(X_train.shape)
print(y_test.shape)

(12203, 52)
(3051,)


In [115]:
noise_type = "class_independent"

fixed_class = 1
fixed_noise = 0.0

model_type = "NN"
uncertainty_type = "forward"
if dataset == "cshock_eicu":
    batch_size = 512
elif dataset == "lungcancer":
    batch_size = 2048
else:
    batch_size = 1024

max_iter = 10000
m = 10
d= X_train.shape[1]
noise_level = 0.2


misspecify = False

y_train = y_train.astype(int)
y_test = y_test.astype(int)

p_y_x_dict =  calculate_prior(y_train, noise_type = noise_type, group=group_train) 


In [116]:
_, T_true = generate_class_independent_noise(y_train, noise_level) #Fixed noise draw


if misspecify == True: #Misspecified T
    pass #TODO
    T_est = None
else: #Correct T
    T_est = T_true



In [117]:
metrics = simulate_noise_and_train_model(m, max_iter,  X_train, y_train, X_test, y_test, p_y_x_dict, T_true = T_true, T_est = T_est, noise_type = noise_type, model_type = model_type, uncertainty_type=uncertainty_type,  fixed_class=fixed_class, fixed_noise=fixed_noise, batch_size = batch_size)


  0%|          | 0/10000 [00:00<?, ?it/s]

Typical Rate:  1.0
Typical Rate:  1.0
Typical Rate:  1.0
Typical Rate:  1.0
DONE


In [118]:
np.mean(metrics.data["BCE"]["clean_test_acc"])

80.79318256309406

In [119]:
np.mean(metrics.data["Ours"]["clean_test_acc"])

80.21632251720749

In [120]:
np.mean(metrics.data["backward"]["clean_test_acc"])

78.56440511307767

In [121]:
np.mean(metrics.data["forward"]["clean_test_acc"])

81.14716486397904

In [122]:
np.mean(metrics.data["backward"]["noisy_train_loss"])

-2.281650845706463